In [1]:
from IPython.display import clear_output

In [2]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt

**Константы для дальнейшей работы**

In [3]:
a = 1
dl = 0.1
dt = 0.09
L = 1
T = 1
T_iter = int(T/dt) + 1
L_iter = int(L/dl) + 1
gamma = (a*dt/dl)**2 

**Рассмотрим тестовую функцию:**

$y(t, x) = t^2(x-l)^2$

**Тогда:**

$y_{tt} = y_{xx} + f(t, x)$

$y_x|_{x=0} = -u(t),\qquad y_x|_{x=0} = 0$

$y|_{t=0} = 0,\qquad\quad\, y_t|_{t=0}=0$

**и**

$u(t) = (2t^2(x-l))|_{x=0} = 2t^2l$

$f(t, x) = 2(x-l)^2 - 2t^2$

In [4]:
def test_function(t, x):
    return 2*((x-L-t)*(x-L+t))

**Тестовое начальное условие**

In [5]:
def test_left_control(t):
    return np.cos(t)

**Создание пространственно-временной сетки**

In [6]:
"""
Инициализация сетки нулями
"""

def make_grid():
    return np.zeros([T_iter, L_iter])

**Функция для отрисовки y(t, x)**

In [7]:
def plot_3D(U):
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(1, 1, 1, projection = '3d')
    xval = np.linspace(0, L, num=T_iter)
    ax.set_xlabel('x', fontsize = 20)
    yval = np.linspace(0, T, num=L_iter)
    ax.set_ylabel('t', fontsize = 20)
    ax.set_zlabel('y(t, x)', fontsize = 20)
    x, y = np.meshgrid(yval,xval)
    z = U
    surf = ax.plot_surface(x, y, z, rstride = 1, cstride = 1, cmap = cm.plasma) 

**Разностная схема крест для уравнения:** 

$y_{tt} = y_{xx} + f(t, x)$, при

$f(t, x) = 2(x-l)^2-2t^2$ 

**Значение, которое вычисляем** $ = Y[t+1, x]$

**Центр в точке** $Y[t,x]$

**Левое граничное условие в точке** $Y[t+1, 0]$ 

**Правое граничное условие в точке** $Y[t+1, L]$

In [331]:
def approximate_solution_cross_straight(Y, control):
    
    for t in range(1, T_iter-1):
        for x in range(1, L_iter-1):

            Y[t+1, x] = 2*Y[t, x] - Y[t-1, x]+gamma*(Y[t, x-1] - 2*Y[t, x] + Y[t, x+1])
            
        Y[t+1, 0] = -dl*control[t] + Y[t+1, 1]
        Y[t+1, L_iter-1] = Y[t+1, L_iter-2]

    return Y

In [342]:
# Задаем оптимальное управление u = cos(t) в узлах сетки
u = np.array([(dt*i,np.cos(dt*i)) for i in range(0, T_iter)])
print('points  = {}\ncos = u = {}'.format(u[:,0],np.round(u[:,1],2)))
u = u[:,1]

points  = [0.   0.09 0.18 0.27 0.36 0.45 0.54 0.63 0.72 0.81 0.9  0.99]
cos = u = [1.   1.   0.98 0.96 0.94 0.9  0.86 0.81 0.75 0.69 0.62 0.55]


In [350]:
PLOT = approximate_solution_cross_straight(make_grid(), u)
PLOT[:5,:5]
# plot_3D(PLOT)

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.09959527,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.17905654, -0.08067217,  0.        ,  0.        ,  0.        ],
       [-0.27206831, -0.17569122, -0.06534446,  0.        ,  0.        ]])

In [355]:
# Финальное состояние (цель) должно быть равно: y(T,x)= -sin(x-1)
goal_1 = np.array([np.sin(dt*(T_iter-1)-dl*x) for x in range(0, L_iter)])
f1_app = PLOT[T_iter-1, :]
print('goal     = {}\nsolution = {}'.format(np.round(goal_1,2), np.round(f1_app,2)))

goal     = [ 0.84  0.78  0.71  0.64  0.56  0.47  0.38  0.29  0.19  0.09 -0.01]
solution = [-0.77 -0.71 -0.64 -0.56 -0.48 -0.39 -0.3  -0.21 -0.09 -0.01 -0.01]


In [340]:
def accurate_solution(y_t):
    Y = np.zeros([T_iter, L_iter])

    for t in range(0, T_iter):
        for x in range(0, L_iter):
            Y[t, x] = y_t(t*dt, x*dl)
            
    return Y

In [ ]:
def get_goal_f1():
    l = np.zeros(L_iter)
    for x in range(L_iter):
        l[x] = -np.sin(dl*x - 1)
        
    return np.array(l)

get_goal_f1()

**Ещё тесты**

# 1
# $u(t) = \sin^2(t)+\cos^2(t)$
# $y = t-x, t>x$

In [ ]:
def control1(t):
    return (np.sin(t))**2 + (np.cos(t))**2

In [ ]:
def y1(t, x):
    if t - x >0:
        return t-x
    return 0

In [ ]:
grid = make_grid()
PLOT = approximate_solution_cross_straight(grid, control1, 0)

In [ ]:
# plot_3D(PLOT)

In [ ]:
# goal = accurate_solution(y1)
# plot_3D(goal)

# 2
# $u(t) = \sin^4(t)+\cos^4(t)$
# $y = -0.0625\cdot(\sin(4(x-t))+12(x-t)), t>x$

In [ ]:
def control2(t):
    return (np.cos(t)**4)+(np.sin(t)**4)

In [ ]:
def y2(t, x):
    if t - x >0:
        return (-1/16)*(np.sin(4*(x-t)) + 12*(x-t))
    return 0

In [ ]:
grid = make_grid()
# PLOT = approximate_solution_cross_straight(grid, control2, 0)
# plot_3D(PLOT)

In [ ]:
# goal = accurate_solution(y2)
# plot_3D(goal)

# 3
# $u(t) = \cos(t+T)$
# $y = -\sin(x-t-T)-\sin(T), t>x$

In [ ]:
def control3(t):
    return np.sin(t)

In [ ]:
def y3(t, x):
    if t - x > 0:
        return 1 - np.cos(x-t)
    return 0

In [ ]:
grid = make_grid()
# PLOT = approximate_solution_cross_straight(grid, control3, 0)
# plot_3D(PLOT)

In [ ]:
# goal = accurate_solution(y3)
# plot_3D(goal)

In [ ]:
def plot_3D_conj(U):
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(1, 1, 1, projection = '3d')
    xval = np.linspace(0, T, num=T_iter)
    ax.set_xlabel('t', fontsize = 20)
    yval = np.linspace(0, L, num=L_iter)
    ax.set_ylabel('x', fontsize = 20)
    ax.set_zlabel('psi(t, x)', fontsize = 20)
    x, y = np.meshgrid(yval,xval)
    z = U
    surf = ax.plot_surface(y, x, z, rstride = 1, cstride = 1, cmap = cm.plasma) 

# Сопреяженная задача 1
# $\frac{\psi_{T,x} - \psi_{T-1, x}}{dt}=-v(x)$
# Возьмем $v(x) = -cos(x)$

In [274]:
def approximate_solution_cross_conjugate_1(PSI, control):

    
    for t in range(T_iter-2, 0, -1):
        for x in range(1, L_iter-1):

            PSI[t-1, x] = 2*PSI[t, x] - PSI[t+1, x] + gamma*(PSI[t, x-1] - 2*PSI[t,x] + PSI[t, x+1])
            PSI[T_iter-1, x+1] = -dt*control[x+1] + PSI[T_iter-2, x+1]
        
        PSI[t-1, 0] = PSI[t-1, 1] 
        PSI[t-1, L_iter-1] = PSI[t-1, L_iter-2]
        
    return PSI

In [360]:
v = [1. for x in range(0, L_iter)]

In [361]:
grid = make_grid()
PSI1 = approximate_solution_cross_conjugate_1(grid, v)
# plot_3D_conj(PSI1)

In [362]:
PSI1[:,0]

array([0.79937937, 0.71072565, 0.61978223, 0.5292795 , 0.44147386,
       0.34874092, 0.25956045, 0.173502  , 0.0729    , 0.        ,
       0.        , 0.        ])

In [363]:
def v_acc1(t, x):
    def res(t,x):
        return (-1/2)*(np.sin(x+t-1)-np.sin(x-t+1))
    if abs(x) < t and t < 2-abs(x):
        return res(t,x) 
    else:
        return res(t,x)

In [365]:
goal = accurate_solution(v_acc1)
goal[:,0]

array([0.84147098, 0.78950374, 0.73114583, 0.66686964, 0.59719544,
       0.52268723, 0.44394811, 0.36161543, 0.27635565, 0.18885889,
       0.09983342, 0.00999983])

In [ ]:
# goal[:, 0]

# Сопреяженная задача 2
# $\psi_{T,x}=w(x)$

In [ ]:
def approximate_solution_cross_conjugate_2(PSI, control):

    for t in range(T_iter-2, 0, -1):
        for x in range(1, L_iter-1):

            PSI[t-1, x] = 2*PSI[t, x] - PSI[t+1, x] + gamma*(PSI[t, x-1] - 2*PSI[t,x] + PSI[t, x+1])
            PSI[T_iter-1, x+1] = dt*control[x+1]
            
        PSI[t-1, 0] = PSI[t-1, 1]
        PSI[t-1, L_iter-1] = PSI[t-1, L_iter-2]
        
    return PSI

In [ ]:
# v = [np.cos(i) for i in np.linspace(0, 1, L_iter)]

# grid = make_grid()
# PSI2 = approximate_solution_cross_conjugate_2(grid, v)

# plot_3D_conj(PSI2)

In [ ]:
# GOAL_acc[T_iter-1, :]

In [ ]:
def v_acc2(t, x, C=3):
    def res(t, x):
#         return (1/(2*C*np.pi))*(np.sin(np.pi*C*x+np.pi*C*t-np.pi*C)-\
#                                 np.sin(np.pi*C*x-np.pi*C*t+np.pi*C))
        return (1/2)*np.sin(x-t)
    
    if t >= -x+1 and t >= x:
        return res(t, x)
    
    elif t <= -x+1 and t <= x:
        return res(t, x)
    
    elif t <= -x+1 and t >= x:
        return res(t, x)
    
    else:
        return res(t, x)

In [ ]:
# goal = accurate_solution(v_acc2)
# plot_3D_conj(goal)

# Сопреяженная задача 3
# $\frac{\psi_{t,L} - \psi_{t, L-1}}{dx} = v(t)$

In [ ]:
def approximate_solution_cross_conjugate_3(PSI, control):
    
    for t in range(1, T_iter-1):
        for x in range(1, L_iter-1):
            PSI[t+1, x] = 2*PSI[t, x] - PSI[t-1, x] + gamma*(PSI[t, x-1] - 2*PSI[t, x] + PSI[t, x+1])
                
        PSI[t+1, L_iter-1] = dl*control[t+1] + PSI[t+1, L_iter-2]
        
    return PSI

In [ ]:
# grid = make_grid()
# v = [np.cos(i) for i in np.linspace(0, 1, T_iter)]
# PSI3 = approximate_solution_cross_conjugate_3(grid, v)
# plot_3D_conj(PSI3)

In [ ]:
def v_acc3(t, x, C=1):
    
    def res(t, x, C):
#         return (1/(np.pi*C))*(np.sin(np.pi*C*x - np.pi*C*t)+np.sin(np.pi*C))
        return -np.sin(1+x-t)

    if t >= 1 - x:
        return res(t, -x, C)
    else:
        return 0
    
# goal = accurate_solution(v_acc3)
# plot_3D_conj(goal)

In [375]:
def SR1(u):
    
    ITERATIONS = 0

    def get_grad(u_current):
        lambda1=1
#         lambda3=1
                
        Y = approximate_solution_cross_straight(make_grid(), u_current)
        
        y1 = Y[T_iter-1, :]   # Значение при заданном управлении = y1(1,x,u)
        f_1 = f1_app          # Цель = f1(1,x,u)
        v1 = y1 - f_1         # v1 для 1й краевой задачи
#         print(np.array(y1), np.array(f_1),sep='\n')
        J1_d = 2*approximate_solution_cross_conjugate_1(make_grid(), v1)[:, 0]
        
#         y3 = Y[:, L_iter-1]          # Значение при заданном управлении = y3(t, L)
#         g = GOAL_acc[:, L_iter-1]    # Цель = g(t, L)
#         v3 = y3 - g                  # v3 для 3й краевой задачи
#         J3_d = 2*approximate_solution_cross_conjugate_3(make_grid(), v3)[:, 0]

        return lambda1*J1_d
    #   _________________________________________________________________________  
    
    H = np.eye(len(u))
    
    eps = 10**(-3)
    alpha = 1
    
    grad = get_grad(u)
    return grad

    while True and ITERATIONS < 10:
        
        norm_grad = np.linalg.norm(grad)
        
        if norm_grad < eps:
            break        

        u_next = u - alpha*H@get_grad(u)
        sk = u_next - u
        u  = u_next

#         clear_output(wait=True)
        print('Method SR1 is on {} iteration, gradient\'s norm = {}'.format(ITERATIONS, norm_grad))
        print('Desired value of gradient`s norm -> {}'.format(eps))
        print('___________________________________________________')
                
        grad_next = get_grad(u_next)
        yk = grad_next - grad
        grad = grad_next
        
        tmp = sk-(H@yk)
        
        H_next = H + (tmp@tmp.T)/(np.dot(tmp, yk))
        H = H_next
        
        ITERATIONS += 1
        
    print('Final grad = {}'.format(grad))
    print('___________________________________________________')
    
    return np.round(np.array(u),2)

In [377]:
# %%time
u_rand = np.random.rand(T_iter)
u_opt = np.array([np.cos(dt*i) for i in range(0, T_iter)])
u = u_opt*0

SR1(u)

array([0.53535361, 0.53482731, 0.51848849, 0.4853715 , 0.44062946,
       0.37428902, 0.29780036, 0.21199353, 0.09301873, 0.        ,
       0.        , 0.        ])

In [378]:
np.array([np.cos(1) - np.cos(t*dt) for t in range(0, T_iter)])

array([-0.45969769, -0.45565043, -0.44354139, -0.42346859, -0.39559452,
       -0.3601448 , -0.31740638, -0.2677252 , -0.21150342, -0.14919613,
       -0.08130766, -0.00838755])

In [ ]:
for i in range(0,T_iter):
    print(np.cos(dl*i))